In [1]:
import pandas as pd
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import dateutil

def parse(line):
    elems = line.split(' ')
    
    obj = {
        'timestamp': dateutil.parser.parse(elems[0])
    }
    for elem in elems[1:]: 
        (key,v) = elem.split(':')
        obj[key] = v
        
    return obj

def extract_byte(x, index):
    def byte_mapper(x):
        if not isinstance(x, basestring):
            return None
        if len(x) > 0:
            return ord(x.decode('hex')[index])
        else:
            return None
    return x.map(byte_mapper)

lines = open('all.txt', "rt").readlines()
column_order = ['timestamp', 'ID1', 'PTYPE', 'SEQ', 'ID2', 'CON', 'B9', 'BLEN', 'MTYPE', 'BODY', 'CRC']
packets = pd.DataFrame(map(lambda x: parse(x), lines), columns=column_order).set_index('timestamp')
packets.head()

,ID1,PTYPE,SEQ,ID2,CON,B9,BLEN,MTYPE,BODY,CRC
timestamp,,,,,,,,,,
2016-06-17 20:23:30.966899,1f014829,POD,25,1f014829,NaN,28,10,1d18,02a82800002b7bff8134,45\n
2016-06-17 20:24:44.634776,1f014829,POD,28,1f014829,NaN,30,10,1d18,02a82800002b7fff0360,ea\n
2016-06-17 20:25:53.328967,1f014829,POD,31,1f014829,NaN,38,10,1d18,02a8a800002b87ff037c,f2\n
2016-06-17 20:48:44.551830,1f014829,POD,02,1f014829,NaN,00,10,1d18,02ad2800002bdfff009c,df\n
2016-06-17 20:48:47.828112,1f014829,POD,05,1f014829,NaN,08,10,1d18,02ad2800002bdfff83c5,cf\n


In [2]:
# Drop packets that have incorrect body length; likely corrupt packets
packets['blen_actual'] = packets['BODY'].map(lambda x: len(x) / 2, 'ignore').fillna(0).astype(np.int)
packets['BLEN'] = packets['BLEN'].map(lambda x: int(x), 'ignore').fillna(0)
ok_big_packets = (packets['blen_actual'] == 23) & (packets['BLEN'] > 23)
no_body_packets = (packets['BLEN'] == 0)
ok_small_packets = (packets['blen_actual'] == packets['BLEN']) & (packets['blen_actual'] > 0)
packets = packets.loc[ok_big_packets | no_body_packets | ok_small_packets]
packets = packets.drop('blen_actual', axis=1)

# Drop sequential duplicates
packets = packets[packets["SEQ"].shift(-1) != packets["SEQ"]]


### Look for body crc
(body crc is different from the 'CRC' column (packet crc) in the dataframe)

We're focused on the last two bytes of the body, as they seem to change like a crc.

* byte -2: bit 7 and bits 1:0 toggle, but 6:2 are always 0
* byte -1: two quiet bits: 6 and 0

In [3]:
# Let's compare packets of the same length, first
packets['BLEN'].value_counts()

0.0      326
10.0     291
3.0      259
31.0       5
24.0       3
12.0       3
205.0      2
21.0       2
6.0        2
126.0      1
50.0       1
32.0       1
29.0       1
23.0       1
Name: BLEN, dtype: int64

### xor each packet against all others, to identify bits that change (bit_deltas)

In [8]:
import itertools

# Compare packets
body_packets = packets.loc[packets['BLEN'] == 3]

# Build raw packet data of everything after sequence, as sequence doesn't appear to
# affect the crc
blen_hex = body_packets["BLEN"].map(lambda x: chr(int(x)).encode('hex'))
raw_hex_data = body_packets["ID2"] + body_packets["B9"] + blen_hex + body_packets["MTYPE"] + body_packets["BODY"]

print raw_hex_data.head()
raw_data = raw_hex_data.map(lambda x: x.decode('hex'))

def xor_strings(s1,s2):
    xored_bytes = [ord(a) ^ ord(b) for a,b in zip(s1,s2)]
    return [s1.encode('hex'),s2.encode('hex')] + list(np.unpackbits(np.array(xored_bytes, dtype=np.uint8)))

combinations = map(lambda x: xor_strings(x[0], x[1]), itertools.combinations(list(raw_data), 2))
bit_deltas = pd.DataFrame(combinations)

#
crc_split_point = -16
bit_cols = list(range(0,len(combinations[0])-2))
body_columns = map(lambda x: ('body', x), bit_cols[:crc_split_point])
crc_columns = map(lambda x: ('crc', x), bit_cols[crc_split_point:])
                
bit_deltas.columns = pd.MultiIndex.from_tuples([('data', 'p1'), ('data', 'p2')] + body_columns + crc_columns)
bit_deltas['num_body_deltas'] = bit_deltas['body'].sum(axis=1)
bit_deltas['num_crc_deltas'] = bit_deltas['crc'].sum(axis=1)

# Yes, this is unwieldy, but necessary to be able to scan columns visually
pd.options.display.max_columns = 200
bit_deltas.head()

timestamp
2016-06-26 20:33:28.749506    1f01482a10030e0100802c
2016-06-26 20:34:22.745483    1f01482a18030e01008322
2016-06-26 20:35:08.728964    1f01482a20030e01000096
2016-06-26 20:37:19.403747    1f01482a28030e01000398
2016-06-26 20:37:52.129892    1f01482a30030e010083be
dtype: object


data                         body                       \
                       p1                      p2    0  1  2  3  4  5  6  7   
0  1f01482a10030e0100802c  1f01482a18030e01008322    0  0  0  0  0  0  0  0   
1  1f01482a10030e0100802c  1f01482a20030e01000096    0  0  0  0  0  0  0  0   
2  1f01482a10030e0100802c  1f01482a28030e01000398    0  0  0  0  0  0  0  0   
3  1f01482a10030e0100802c  1f01482a30030e010083be    0  0  0  0  0  0  0  0   
4  1f01482a10030e0100802c  1f01482a3c030e010001bb    0  0  0  0  0  0  0  0   

                                                                              \
   8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32   
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
4  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   

                                                                              \
  33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57   
0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
1  0  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
2  0  1  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
3  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
4  0  1  0  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   

                                            crc                                \
  58 59 60 61 62 63 64 65 66 67 68 69 70 71  72 73 74 75 76 77 78 79 80 81 82   
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   0  0  0  0  0  0  1  1  0  0  0   
1  0  0  0  0  0  0  0  0  0  0  0  0  0  0   1  0  0  0  0  0  0  0  1  0  1   
2  0  0  0  0  0  0  0  0  0  0  0  0  0  0   1  0  0  0  0  0  1  1  1  0  1   
3  0  0  0  0  0  0  0  0  0  0  0  0  0  0   0  0  0  0  0  0  1  1  1  0  0   
4  0  0  0  0  0  0  0  0  0  0  0  0  0  0   1  0  0  0  0  0  0  1  1  0  0   

                 num_body_deltas num_crc_deltas  
  83 84 85 86 87                                 
0  0  1  1  1  0               1              5  
1  1  1  0  1  0               2              6  
2  1  0  1  0  0               3              7  
3  1  0  0  1  0               1              5  
4  1  0  1  1  1               3              7

In [5]:
# This selects the number of 'collisions', or different frames with same crc.
# It could be used to identify bits not part of crc

#bit_deltas[(bit_deltas["num_body_deltas"] != 0) & (bit_deltas["num_crc_deltas"] == 0)]


In [9]:

# Look at low change rows
ones = bit_deltas[(bit_deltas["num_body_deltas"] == 1)]
map(lambda x: (x, ones.loc[ones[('body',x)] == 1]["crc"].iloc[0:1].values), range(0,ones["body"].shape[1]))
#ones.loc[ones[('body',34)] == 1]["data"]


[(0, array([], shape=(0, 16), dtype=int64)),
 (1, array([], shape=(0, 16), dtype=int64)),
 (2, array([], shape=(0, 16), dtype=int64)),
 (3, array([], shape=(0, 16), dtype=int64)),
 (4, array([], shape=(0, 16), dtype=int64)),
 (5, array([], shape=(0, 16), dtype=int64)),
 (6, array([], shape=(0, 16), dtype=int64)),
 (7, array([], shape=(0, 16), dtype=int64)),
 (8, array([], shape=(0, 16), dtype=int64)),
 (9, array([], shape=(0, 16), dtype=int64)),
 (10, array([], shape=(0, 16), dtype=int64)),
 (11, array([], shape=(0, 16), dtype=int64)),
 (12, array([], shape=(0, 16), dtype=int64)),
 (13, array([], shape=(0, 16), dtype=int64)),
 (14, array([], shape=(0, 16), dtype=int64)),
 (15, array([], shape=(0, 16), dtype=int64)),
 (16, array([], shape=(0, 16), dtype=int64)),
 (17, array([], shape=(0, 16), dtype=int64)),
 (18, array([], shape=(0, 16), dtype=int64)),
 (19, array([], shape=(0, 16), dtype=int64)),
 (20, array([], shape=(0, 16), dtype=int64)),
 (21, array([], shape=(0, 16), dtype=int64))